In [1]:
import tensorflow as tf
import input_data
import numpy as np
import time
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
np.set_printoptions(suppress=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Network Parameters
n_input = 784
n_hidden_1 = 256
n_hidden_2 = 256
n_classes = 10

# Training Parameters
learning_rate = 0.001
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
training_epochs = 10

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y_true = tf.placeholder(tf.float32, [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [3]:
# Create model
def nn(x, weights, biases):
    # Hidden layer 1 with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer 2 with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
#     out = tf.matmul(layer_2, weights['out']) + biases['out']
    out = tf.add(tf.matmul(layer_2, weights['out']),biases['out'])
    return out

In [4]:
# Construct model
y_pred = nn(x, weights, biases)
y_softmax = tf.nn.softmax(y_pred)

# Define loss and optimizer
# type 1:
# loss = tf.reduce_mean(
#     -tf.reduce_sum(y_true * tf.log(y_softmax), reduction_indices=[1]))
# type 2:
# loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
# type 3:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


# Evaluate model
pred = tf.argmax(y_pred, 1)
true = tf.argmax(y_true, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training cycle
start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c = sess.run(
            [optimizer, loss],
            feed_dict={
                x: batch_xs,
                y_true: batch_ys
            })
        ave_cost += c / total_batch
    # Display logs per epoch step
    if epoch_i % 1 == 0:
        train_acc = accuracy.eval(feed_dict={x: batch_xs, y_true: batch_ys})
        test_acc = accuracy.eval(feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels
        })
        print("Epoch:%3d Batch:%5d " % (epoch_i,
                                        batch_i), "train_acc=%.5f" % train_acc,
              "test_acc=%.5f" % test_acc, "train_cost=%.9f" % ave_cost)
end = time.time()
print("Process Time :%.2f s" % (end - start))

# Calculate accuracy
print(
    sess.run(
        accuracy, feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels
        }))
sess.close()

Epoch:  0 Batch:  549  train_acc=0.82000 test_acc=0.85580 train_cost=193.774311458
Epoch:  1 Batch:  549  train_acc=0.92000 test_acc=0.89530 train_cost=42.442684514
Epoch:  2 Batch:  549  train_acc=0.96000 test_acc=0.90720 train_cost=26.576025040
Epoch:  3 Batch:  549  train_acc=0.94000 test_acc=0.91620 train_cost=18.617165884
Epoch:  4 Batch:  549  train_acc=0.99000 test_acc=0.92340 train_cost=13.602172794
Epoch:  5 Batch:  549  train_acc=0.95000 test_acc=0.92670 train_cost=10.126756996
Epoch:  6 Batch:  549  train_acc=0.94000 test_acc=0.93030 train_cost=7.747670076
Epoch:  7 Batch:  549  train_acc=0.94000 test_acc=0.93200 train_cost=5.821446383
Epoch:  8 Batch:  549  train_acc=0.99000 test_acc=0.93680 train_cost=4.332714139
Epoch:  9 Batch:  549  train_acc=0.99000 test_acc=0.93730 train_cost=3.220821974
Process Time :12.38 s
0.9373
